In [45]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
from urllib.error import URLError
import urllib.parse
import ssl

## csv 파일 (영화제목들) 불러와서 리스트로 뽑아내기

In [46]:
movie_lists = []
f = open('movie_name (2).csv', 'r', encoding='euc-kr')
rdr = csv.reader(f)
for line in rdr:
    print(line[0])
    movie_lists.append(line[0])
f.close()    

title1
명량
국제시장
아바타
베테랑
괴물
도둑들
7번방의 선물
암살
광해, 왕이 된 남자
왕의 남자
부산행
해운대
변호인
어벤져스: 에이지 오브 울트론
겨울왕국
인터스텔라
검사외전
관상
아이언맨 3


In [47]:
movie_lists = movie_lists[1:]
movie_lists

['명량',
 '국제시장',
 '아바타',
 '베테랑',
 '괴물',
 '도둑들',
 '7번방의 선물',
 '암살',
 '광해, 왕이 된 남자',
 '왕의 남자',
 '부산행',
 '해운대',
 '변호인',
 '어벤져스: 에이지 오브 울트론',
 '겨울왕국',
 '인터스텔라',
 '검사외전',
 '관상',
 '아이언맨 3']

# 영화 하나씩 돌면서 처리해주기

In [48]:
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query="
context = ssl._create_unverified_context()
for movie in movie_lists:
    
    print(movie)
    # 영화 링크 찾기
    name = url+ urllib.parse.quote_plus("영화 ") + urllib.parse.quote_plus(movie)
    req = urllib.request.Request(name, headers={'User-Agent': 'Mozilla/5.0'})

    try:
        html = urlopen(req,context=context).read()
    except URLError as e:
        print(e)
        continue
    
    bsObj = BeautifulSoup(html,"html.parser")
    
    link = bsObj.find("a",{"class":"sh_movie_link"}).get('href')

    # 영화 링크로 들어가기
    ## 그전에 치환(영화배우 출력위해)
    link = link.replace("basic","detail")
    req = urllib.request.Request(link, headers={'User-Agent': 'Mozilla/5.0'})
    try:
        html = urlopen(req,context=context)
    except URLError as e:
        print(e)
        continue
    bsObj = BeautifulSoup(html,"html.parser")
    
    # 주연 뽑아내기
    mains = []
    heroes = bsObj.findAll('div',{'class':'p_info'})
    for hero in heroes:
        part = hero.find('em',{'class':'p_part'}).get_text()
        if part == '주연':
            mains.append(hero.find('a',{'class':'k_name'}).get('title'))
    print(mains)        
            
    # 평점 뽑아내기
    score = bsObj.find('span',{'class':'st_on'}).get_text()
    score = score.split('평점')[1].strip()
    if score == '없음':
        score = 0
    else:
        score = score[:-1]
    print(score)
    
    # 기대지수 뽑아내기
    ## 그전에 치환(기대지수 출력위해)
    link = link.replace("detail","point")
    req = urllib.request.Request(link, headers={'User-Agent': 'Mozilla/5.0'})
    try:
        html = urlopen(req,context=context)
    except URLError as e:
        print(e)
        continue
    bsObj = BeautifulSoup(html,"html.parser")
    
    expectation = bsObj.find('span',{'class':'exp_cnt'}).get_text()[5:].replace(',','')
    print(expectation)
    print('-----')
    
    # csv 로 만들기
    with open('./files/'+movie+'.csv','w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        for main in mains:
            writer.writerow([main,score,expectation])

명량
['최민식', '류승룡', '조진웅']
8.87
13639
-----
국제시장
['황정민', '김윤진', '오달수', '정진영']
9.16
12784
-----
아바타
['샘 워싱턴', '조 샐다나', '시고니 위버', '스티븐 랭']
0
0
-----
베테랑
['황정민', '유아인', '유해진', '오달수']
9.24
9952
-----
괴물
['송강호', '변희봉', '박해일', '배두나', '고아성']
0
0
-----
도둑들
['김윤석', '김혜수', '이정재', '전지현', '임달화', '김해숙', '오달수', '김수현', '이신제', '증국상']
0
0
-----
7번방의 선물
['류승룡', '박신혜', '갈소원', '오달수', '박원상', '김정태', '정만식', '김기천']
0
0
-----
암살
['전지현', '이정재', '하정우']
9.10
20846
-----
광해, 왕이 된 남자
['이병헌', '류승룡', '한효주']
0
0
-----
왕의 남자
['감우성', '정진영', '강성연', '이준기']
0
0
-----
부산행
['공유', '정유미', '마동석', '김수안', '김의성', '최우식', '안소희']
8.59
62952
-----
해운대
['설경구', '하지원', '박중훈', '엄정화']
0
0
-----
변호인
['송강호', '김영애', '오달수', '곽도원', '임시완']
9.29
0
-----
어벤져스: 에이지 오브 울트론
['로버트 다우니 주니어', '크리스 헴스워스', '마크 러팔로', '크리스 에반스', '스칼렛 요한슨', '제레미 레너', '돈 치들', '제임스 스페이더', '사무엘 L. 잭슨']
8.59
96176
-----
겨울왕국
['크리스틴 벨', '이디나 멘젤']
9.20
0
-----
인터스텔라
['매튜 맥커너히', '앤 해서웨이', '마이클 케인', '제시카 차스테인']
9.11
11513
-----
검사외전
['황정민', '강동원']
8.56
46891
-----
관상
['송강호', '이정재', '백